In [1]:
import torch
import numpy as np
import pandas as pd
import time
import sys
import pickle

In [2]:
site = 'C'

In [3]:
# get parent directory
os.chdir("..")

# import file
from network import Net
from learning import train

# get higher level parent directory
os.chdir("..")

directory = 'Data_clean'
data_name = site + '_data_pretrain.csv'
scaler_nameX = site + "_scalerX.pkl"
scaler_nameY = site + "_scalerY.pkl"

# load pretrain data and scaler functions 
data_pretrain = pd.read_csv(os.path.join(os.getcwd(), directory, data_name), index_col = 0)
scalerX = pickle.load(open(os.path.join(os.getcwd(), directory, scaler_nameX),'rb'))
scalerY = pickle.load(open(os.path.join(os.getcwd(), directory, scaler_nameY),'rb'))

# reset to current directory
fd = sys.path[0]
os.chdir(fd)

In [4]:
# separate X and y, and apply standardscaler transform (normalize and scale to unit variance)
X_pretrain = data_pretrain.iloc[:, 5:-1].values
X_pretrain = scalerX.transform(X_pretrain)

y_pretrain = data_pretrain.iloc[:, -1:].values
y_pretrain = scalerY.transform(y_pretrain)

### Pretraining

In [5]:
# load hyperparameter settings
model = torch.load( sys.path[0] + '/hparams.pth')

In [6]:
# # define quantiles and hyperparameters
q_median = 0.5
q_upper = 0.975
q_lower = 0.025
dims = model['dims']
lr = model['lr']
batch_size = int(model['batch_size'])
epoch = model['epoch']

print("lr:", lr)
print("batch size:", batch_size)
print("epoch:", epoch)

lr: 0.005
batch size: 1000
epoch: 20


In [7]:
# # initialize network 
net_median = Net(dims = dims)
net_upper = Net(dims = dims)
net_lower = Net(dims = dims)

In [8]:
%%time
# Train
net_median, median_state_dict = train(X_train=X_pretrain, y_train=y_pretrain, quantile=q_median, net=net_median, 
                                     lr=lr, batch_size=batch_size, epoch=epoch)
net_upper, UQ_state_dict = train(X_train=X_pretrain, y_train=y_pretrain, quantile=q_upper, net=net_upper, 
                                 lr=lr, batch_size=batch_size, epoch=epoch)
net_lower, LQ_state_dict = train(X_train=X_pretrain, y_train=y_pretrain, quantile=q_lower, net=net_lower, 
                                 lr=lr, batch_size=batch_size, epoch=epoch)

Wall time: 2min 5s


### Save trained network

In [15]:
# create dictionary to store the trained weights
pretrain_state_dict = {'median': net_median.state_dict(), 
                       'UQ': net_upper.state_dict(),
                       'LQ': net_lower.state_dict()}

In [16]:
# save the dictionary
# torch.save(pretrain_state_dict, sys.path[0] + '/pretrain.pth')